# Visualize some multi-channel fluorescence images fro the purposes of figure making

In [6]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd

import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
from scipy import stats
from scipy.spatial import distance
import numpy as np
from scipy import interpolate
from pathlib import Path
import matplotlib as mpl

from bsccm import BSCCM

bsccm = BSCCM(str(Path.home()) + '/BSCCM_local/BSCCM/')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec

cmaps = [LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [0, 0, 1]]),
         LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [0, 1, 0]]),
         LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [0.5, 0.5, 0]]),
         LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [1, 0, 0]]),
         LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [0.5, 0, 0.5]]),
         LinearSegmentedColormap.from_list('testmap', [[0, 0, 0], [0, 0.5, 0.5]])]

shape = (40,40)

indices = bsccm.get_indices(batch=0, antibodies='all', 
                            shuffle=True)[:np.prod(shape)].reshape(shape)

colors = [cmap, cmap2]
alphas = [1, 0.5]
images = []

cols = []
for i in range(shape[0]):
    rows = []
    for j in range(shape[1]):
        channels = []
        for c_index, channel in enumerate(bsccm.fluor_channel_names):
            channels.append(bsccm.read_image(indices[i,j], contrast_type='fluor', channel=channel))            
        rows.append(np.stack(channels, axis=-1))
    cols.append(rows)
images = np.array(cols)
        
ch_min = np.min(images, axis=(0,1,2,3))
ch_max = np.max(images, axis=(0,1,2,3))

rescaled = (images - ch_min) / (ch_max - ch_min)

mapped = [cmaps[i](rescaled[..., i]) for i in range(6)]
composite = np.sum(mapped, axis=0)[..., :3]
composite[composite > 1] = 1

fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(*shape)
gs.update(wspace=0.0, hspace=0.0) # set the spacing between axes. 

for i in range(shape[0]):
    for j in range(shape[1]):
        ax = plt.subplot(gs[i, j])
        ax.imshow(composite[i, j])
        ax.set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
plt.savefig('/home/henry/leukosight_data/figures/demixing/fluor_cell_montage_huge.pdf', 
            transparent=True)